In [2]:
# Knowledge-graph of EU-ToxRisk TempOSeq data

# KG for summary data (i.e. concentration independent data)
# Since we have uploaded processed temposeq data and not the summary data,
# we first have to convert it to sort of summary data. This means that we will
# consider only genes with a significant fold change (quick filter based on logFC and p-adj).
# In any case summary data would be reported only significantly expressed probes.

# We aim to use the KG structure of biobricks-OKG (https://github.com/biobricks-ai/biobricks-okg?tab=readme-ov-file)
# Additionally, we add connections between the genes and pathways in the same fashion as Pubchem RDF (https://pubchem.ncbi.nlm.nih.gov/docs/rdf-intro#section=PubChemRDF-Graphs)

# Summary of EUTR-OKG
#   <assay> (study):
#       - has_measure_group <measure_group>
#   <measure_group>:
#       - has_participant <compound>
#       - has_specified_output <endpoint>
#   < endpoint>:
#       - 
#       - has unit <unit>
#   compound participates_in measure_group
#   compound has_identifier CAS/InChI

In [3]:
!pip install rdflib
!pip install rdflib-hdt
#!biobricks install wikipathways
#!biobricks install eutoxrisk-temposeq

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
#import biobricks as bb
from rdflib import Graph, Namespace
from rdflib.plugins.stores import sparqlstore
from rdflib_hdt import HDTStore
from rdflib.namespace import RDF, XSD, DCTERMS
import rdflib
from tqdm import tqdm

In [5]:
# Connect studies, compounds and temposeq dataset id
df_overview = pd.read_csv("../download/overview.csv")
df_overview.columns

# Connect assays and compounds: keep only compounds at the highest concentration with that study
#compound_to_assay = df_overview[['Treatment compound', 'Study']].drop_duplicates().set_index("Treatment compound")['Study'].to_dict()
#compound_to_assay

Index(['Study', 'Treatment compound', 'Treatment concentration',
       'Treatment timepoint', 'Control compound', 'Control concentration',
       'Control timepoint', 'Filename', 'Case study', 'Index in compound list',
       'Comments', 'Parent compound', 'Harris's Preferred Name (ChEMBL/ECHA)',
       'SMILES', 'InChI key', 'CAS number', 'Starting substance',
       'Barbara's prefered name', 'Alternative name',
       'Starting substance SMILES', 'Starting substance CAS number',
       'Supplier', 'Cat no', 'Lot no', 'Form', 'Storage pure compound',
       'Solvent', 'Stock concentration', 'Stock concentration unit',
       'Stock aliquot storage', 'Purity', 'Molar weight', 'Molecular formula',
       'MDLNUM', 'Code', 'Dataset id'],
      dtype='object')

In [6]:
inchi_to_compound_dict = df_overview[['Treatment compound', 'InChI key']].set_index("InChI key")['Treatment compound'].to_dict()
inchi_to_compound_dict_without_underscores = df_overview[['Treatment compound', 'InChI key']].set_index("InChI key")['Treatment compound'].str.replace('_', ' ').to_dict()

In [7]:
# Connect genes and pathways
df_pathways = pd.read_csv("../download/pathways.csv")

In [25]:
# compound to gene relationship
measure_groups = []

# Consider only the highest concentration of compound
df_overview_highest_conc = df_overview.sort_values(by=['Study', 'Treatment compound', 'Treatment concentration']).drop_duplicates(subset=['Study', 'Treatment compound'], keep='last')

for _, row in tqdm(df_overview_highest_conc.iterrows()):
    dataset_id = row['Dataset id']
    d = row.to_dict()

    try:
        data_frame = pd.read_csv("../download/temposeq/{0}.csv".format(dataset_id))
        
        LOG_FOLD_CHANGE_THRESHOLD = 2
        ADJUSTED_P_VALUE_THRESHOLD = 0.05

        is_positive_significant_fold_change = (data_frame['logFC'] > LOG_FOLD_CHANGE_THRESHOLD)
        is_negative_significant_fold_change = (data_frame['logFC'] < -LOG_FOLD_CHANGE_THRESHOLD)
    
        is_significant_p_value = data_frame['padj'] < ADJUSTED_P_VALUE_THRESHOLD
    
        # positive deregulated genes
        for gene in data_frame[is_positive_significant_fold_change & is_significant_p_value]['SYMBOL'].tolist():
            d['Compound'] = "inchikey:{0}".format(row['InChI key'])
            d['Assay'] = "study:{0}".format(row['Study'])
            d['Gene'] = "gene:{0}".format(gene)
            d['Endpoint'] = "endpoint:{0}".format(len(measure_groups))
            d['Endpoint_value'] = "positive"
            measure_groups.append(d)

        # negtive deregulated genes
        for gene in data_frame[is_negative_significant_fold_change & is_significant_p_value]['SYMBOL'].tolist():
            d['Compound'] = "inchikey:{0}".format(row['InChI key'])
            d['Assay'] = "study:{0}".format(row['Study'])
            d['Gene'] = "gene:{0}".format(gene)
            d['Endpoint'] = "endpoint:{0}".format(len(measure_groups))
            d['Endpoint_value'] = "negative"
            measure_groups.append(d)
    except:
        pass

0it [00:00, ?it/s]

373it [00:04, 87.27it/s] 


In [26]:
df_overview_highest_conc.columns

Index(['Study', 'Treatment compound', 'Treatment concentration',
       'Treatment timepoint', 'Control compound', 'Control concentration',
       'Control timepoint', 'Filename', 'Case study', 'Index in compound list',
       'Comments', 'Parent compound', 'Harris's Preferred Name (ChEMBL/ECHA)',
       'SMILES', 'InChI key', 'CAS number', 'Starting substance',
       'Barbara's prefered name', 'Alternative name',
       'Starting substance SMILES', 'Starting substance CAS number',
       'Supplier', 'Cat no', 'Lot no', 'Form', 'Storage pure compound',
       'Solvent', 'Stock concentration', 'Stock concentration unit',
       'Stock aliquot storage', 'Purity', 'Molar weight', 'Molecular formula',
       'MDLNUM', 'Code', 'Dataset id'],
      dtype='object')

In [27]:
g = rdflib.Graph()
context = []

# namespaces
nm = g.namespace_manager

dce, prefix = rdflib.Namespace("http://purl.org/dc/elements/1.1/"), "dce"
nm.bind(prefix, dce)

sio, prefix = rdflib.Namespace("http://semanticscience.org/resource/SIO/"), "sio"
nm.bind(prefix, sio)

cheminf, prefix = rdflib.Namespace("http://semanticscience.org/resource/CHEMINF/"), "cheminf"
nm.bind(prefix, cheminf)

wp, prefix = rdflib.Namespace("http://vocabularies.wikipathways.org/wpTypes#"), "wp"
nm.bind(prefix, wp)

bp, prefix = rdflib.Namespace("http://www.biopax.org/release/biopax-level3.owl#"), "bp"
nm.bind(prefix, bp)

bao, prefix = rdflib.Namespace("http://www.bioassayontology.org/bao#"), "bao"
nm.bind(prefix, bao)

metago, prefix = rdflib.Namespace("http://model.geneontology.org/"), "metago"
nm.bind(prefix, metago)

rdfs, prefix = rdflib.Namespace("http://www.w3.org/2000/01/rdf-schema#"), "rdfs"
nm.bind(prefix, rdfs)

edam, prefix = rdflib.Namespace("http://edamontology.org/"), "edam"
nm.bind(prefix, edam)

obo, prefix = rdflib.Namespace("http://purl.obolibrary.org/obo/"), "obo"
nm.bind(prefix, obo)

# Graph is built from triplets of (subject, predicate, object)

# Assays (studies)
for study in df_overview['Study'].unique():
    
    # Assay is of type bioassay
    s = rdflib.URIRef("study:{0}".format(study))
    p = RDF.type
    o = bao.BAO_0000015
    g.add((s, p, o))


# Compounds (and their identifiers)
compounds = df_overview.set_index("Treatment compound")[['CAS number', 'InChI key']]
for compound_name, row in compounds.iterrows():

    # Compound is of type chemical entity
    s = rdflib.URIRef("inchikey:{0}".format(row['InChI key']))
    p = RDF.type
    o = cheminf.CHEMINF_000000
    g.add((s, p, o))

    # Compound has label compound name
    p = rdfs.label
    o = rdflib.Literal(compound_name, datatype=XSD.string)
    g.add((s, p, o))

    # Compound has identifier CAS number
    p = edam.has_identifier
    o = rdflib.URIRef("cas:{0}".format(row['CAS number']))
    g.add((s, p, o))

    # CAS number if of type CAS number
    p = RDF.type
    o2 = cheminf.CHEMINF_000446
    g.add((o, p, o2))

    # CAS number has label compound name
    p = rdfs.label
    o2 = rdflib.Literal(compound_name, datatype=XSD.string)
    g.add((o, p, o2))

    # CAS number source is "CAS"
    p = dce.source
    o2 = rdflib.Literal("CAS", datatype=XSD.string)
    g.add((o, p, o2))

    # Compound has identifier InChI key
    p = edam.has_identifier
    o = rdflib.URIRef("inchikey:{0}".format(row['InChI key']))
    g.add((s, p, o))
    
    # InChI key if of type InChI key
    p = RDF.type
    o2 = cheminf.CHEMINF_000059
    g.add((o, p, o2))

    # InChI key has label compound name
    p = rdfs.label
    o2 = rdflib.Literal(compound_name, datatype=XSD.string)
    g.add((o, p, o2))



# Genes
for gene_symbol in df_pathways.Genes.unique():
    # Gene is of type gene
    s = rdflib.URIRef("gene:{0}".format(gene_symbol))
    p = RDF.type
    o = sio.SIO_010035
    g.add((s, p, o))

    # Gene has gene symbol gene_symbol
    p = bao.BAO_0002870
    o = rdflib.Literal(gene_symbol, datatype=XSD.string)  # Pubchem has this somehwat diferent - not understood how/why
    g.add((s, p, o))


# Pathways
for pathway_title in df_pathways.Term.unique():
    pathway_name = pathway_title.replace(" ", "-")

    # Pathway is of type pathway
    s = rdflib.URIRef("pathway:{0}".format(pathway_name))
    p = RDF.type
    o = bp.Pathway
    g.add((s, p, o))

    # Pathway has title 
    p = DCTERMS.title
    o = rdflib.Literal(pathway_title, datatype=XSD.string)
    g.add((s, p, o))

# Gene-Pathway relationship
for _, row in df_pathways.iterrows():
    gene_symbol = row['Genes']
    pathway_title = row['Term']
    pathway_name = pathway_title.replace(" ", "-")

    # Gene participates in pathway
    s = rdflib.URIRef("gene:{0}".format(gene_symbol))
    p = obo.RO_0000056
    o = rdflib.URIRef("pathway:{0}".format(pathway_name))
    g.add((s, p, o))

# Measure groups
for i in range(len(measure_groups)):
    
    # Study/Assay has measure group
    study = rdflib.URIRef(measure_groups[i]['Assay'])
    p = bao.BAO_0000209
    measure_group = rdflib.URIRef("measure_group:{0}".format(i))
    g.add((study, p, measure_group))

    # Meaure group is of type measure group
    p = RDF.type
    o = bao.BAO_0000040
    g.add((measure_group, p, o))

    # Measure group has participant compound
    p = obo.RO_0000057
    compound = rdflib.URIRef(measure_groups[i]['Compound'])
    g.add((measure_group, p, compound))

    # Compound participates in measure group
    p = obo.RO_0000056
    g.add((compound, p, measure_group))

    # Measure group has participant gene
    p = obo.RO_0000057
    gene = rdflib.URIRef(measure_groups[i]['Gene'])
    g.add((measure_group, p, gene))

    # Measure group has specified output endpoint
    p = obo.OBI_0000299
    endpoint = rdflib.URIRef(measure_groups[i]["Endpoint"])
    g.add((measure_group, p, endpoint))

    # Endpoint is of type fold change 
    p = RDF.type
    o = bao.BAO_0000193
    g.add((endpoint, p, o))

    # Endpoint has value 
    p = sio.SIO_000300
    endpoint_value = rdflib.Literal(measure_groups[i]["Endpoint_value"], datatype=XSD.string)
    g.add((endpoint, p, endpoint_value))

    # Endpoint is about compound
    p = obo.IAO_0000136
    g.add((endpoint, p, compound))
    
    # Endpoint is about gene
    g.add((endpoint, p, gene))

print("Graph has {0} nodes.".format(len(g.all_nodes())))
print("Graph has {0} statements.".format(len(g)))

Graph has 28811 nodes.
Graph has 152208 statements.


In [16]:
measure_groups[0]

{'Study': 'EUT101',
 'Treatment compound': 'Amino_nitroanilino_ethanol',
 'Treatment concentration': 180.0,
 'Treatment timepoint': 24,
 'Control compound': 'DMSO03',
 'Control concentration': 0.0,
 'Control timepoint': 24,
 'Filename': 'EUT101_Amino_nitroanilino_ethanol_180_24_DMSO03_0_24.csv',
 'Case study': 'CS11',
 'Index in compound list': 31,
 'Comments': nan,
 'Parent compound': '2-(4-amino-2-nitroanilino)ethanol',
 "Harris's Preferred Name (ChEMBL/ECHA)": nan,
 'SMILES': "{'original': 'Nc1ccc(NCCO)c([N+](=O)[O-])c1', 'canonical': 'Nc1ccc(NCCO)c([N+](=O)[O-])c1'}",
 'InChI key': 'GZGZVOLBULPDFD-UHFFFAOYSA-N',
 'CAS number': '4-1-2871',
 'Starting substance': '2-(4-amino-2-nitroanilino)ethanol',
 "Barbara's prefered name": '2-(4-amino-2-nitroanilino)ethanol',
 'Alternative name': '2-((4-AMino-2-nitrophenyl)aMino)ethanol',
 'Starting substance SMILES': "{'original': '[N+](=O)([O-])c1c(ccc(c1)N)NCCO', 'canonical': 'Nc1ccc(NCCO)c([N+](=O)[O-])c1'}",
 'Starting substance CAS number':

In [28]:
q = """
PREFIX sio: <http://semanticscience.org/resource/SIO/>
PREFIX cheminf: <http://semanticscience.org/resource/CHEMINF/>

SELECT ?inchikey ?compound
WHERE {
    ?p rdf:type cheminf:CHEMINF_000399 .
    ?p sio:SIO_000300 ?inchikey .
    ?p sio:SIO_000011 ?compound .
    
}
"""


results = g.query(q)
print(len(results))

for r in results:
    print(str(r))

0
